In [5]:
# -- IMPORTS -- #
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input,decode_predictions
import keras.preprocessing.image as kerasImage
import keras.backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# -- FORWARD TOP LAYER -- #
def forward(rule):
    if rule.layer.__class__.__name__ == 'Dense':
        return K.dot(rule.act,rule.weights[0])
    elif rule.layer.__class__.__name__ == 'MaxPooling2D':
        # mirar si te pool_size i strides
        pooled = K.pool2d(a, pool_size=rule.layer.pool_size, strides=strides[1:-1], padding='valid', pool_mode='max')
        return K.maximum(pooled,0.)

In [6]:
# -- GRADPROP (BACKPROP) TOP LAYER -- #

Instructions for updating:
Colocations handled automatically by placer.
Dense


In [ ]:
# -- Z RULE -- #
class ZRule(object):
    def __init__(self,topLayer,subLayer):
        self.layer = topLayer
        self.weights = topLayer.get_weights()
        self.flag = True if not self.weigths else False
        self.act = subLayer.output
    def run(self,R,ignoreBias=True):
        Z = forward(self)+K.epsilon()
        if not ignoreBias and not self.flag:
            Z += self.weigths[1]
        S = R/Z
        C = gradprop(S)
        R = self.act*C
        return R

In [ ]:
# -- Z+ RULE -- #
class ZPlusRule(object):
    def __init__(self,topLayer,subLayer):
        self.layer = topLayer
        self.weights = topLayer.get_weights()
        self.flag = True if not self.weight else False
        self.act = subLayer.output
    def run(self,R,ignoreBias=True):
        if not self.flag:
            self.weights[0] = K.maximum(self.weights[0],0.)
        Z = forward(self)+K.epsilon()
        if not ignoreBias and not self.flag:
            Z += self.weigths[1]
        S = R/Z
        C = gradprop(S)
        R = self.act*C
        return R

In [ ]:
# -- Z- RULE -- #

In [ ]:
# -- Zb RULE -- #

In [ ]:
# -- Zab RULE -- #

In [22]:
# -- MAIN -- #